In [1]:
#Importamos librerías
import pandas as pd 
from sklearn.preprocessing import StandardScaler
import numpy as np
from xgboost import XGBRegressor
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import linear_model
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import GridSearchCV


In [2]:
#Importamos base de datos
df = pd.read_csv("base_datos_2008.csv")

In [3]:
#Se debe utilizar solo datos que tengan valor en la variable dependiente
df= df.dropna(subset=["ArrDelay","Distance","TaxiIn","TaxiOut","DepDelay"])
df=df.sample(frac=1).head(1000)

In [4]:
X=df[["Distance","TaxiIn","TaxiOut","DepDelay","AirTime"]]
Y=df["ArrDelay"]

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2,random_state=10)

scaler=StandardScaler()
#Siempre escalar con conjunto de train
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [8]:
#Va a explorar entre la combinación de los parámetros
parametros={
    "alpha":[0.000001 , 0.0001 , 0.01 ,0.1 ],#parametro de regularizacion 
    "hidden_layer_sizes":[2,5,10,50,100],#numero de capas ocultas
    "solver":("lbfgs","adam"),#funcion de optimizacion
    "learning_rate":("constant","adaptative"),#como aprende el algoritmo
}

nn = MLPRegressor(warm_start=True, max_iter=100000) #Defino un modelo
clf = GridSearchCV(nn , parametros , cv=5, n_jobs=-1) #cross validation de 5 bloques y paralelizado (todos los núcleos - 1)
clf.fit(X,Y)



C:\Users\Diego\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
200 fits failed out of a total of 400.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
200 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\Diego\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\Diego\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 752, in fit
    return self._fit(X, y, incremental=False)
  File "C:\Users\Diego\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py", line 384, in _fit
    self._validate_hyp

GridSearchCV(cv=5, estimator=MLPRegressor(max_iter=100000, warm_start=True),
             n_jobs=-1,
             param_grid={'alpha': [1e-06, 0.0001, 0.01, 0.1],
                         'hidden_layer_sizes': [2, 5, 10, 50, 100],
                         'learning_rate': ('constant', 'adaptative'),
                         'solver': ('lbfgs', 'adam')})

In [10]:
print(clf.best_params_)

{'alpha': 1e-06, 'hidden_layer_sizes': 100, 'learning_rate': 'constant', 'solver': 'lbfgs'}


In [13]:
nn2=clf.best_estimator_

In [18]:
nn2.fit(X,Y)
nn2.predict(X)
r2_score(Y, nn2.predict(X))

0.9415252189905144